This is a streamlined solution without any data augmentation. The model has two layers, which build upon each other and one layer uses CPU platform while the other uses GPU platform. We decided that the easiest will be to split this into three kernels, which will be three steps. Please run these three kernels in sequence one after the other as described below.
 

(1) Liverppol_LGBM_oofs

(1.a) This kernal has to be run first.

(1.b) This kernal uses competition data and the public dataset Data Without Drift as input.

(1.c) This kernal uses an LGBM based model to make baseline predictions. 

(1.d) This kernal runs on a CPU platform.

(1.e) The runtime for this kernal is approximately 1.7 hrs (it took us 6133.8 sec).

(2) Liverpool_NoiseRemoval

(2.a) This kernal has to be run second.

(2.b) This kernal takes output from the previous kernal as input.

(2.c) This kernal removes 50Hz noise from the signal and we use that clean signal in the next step.

(2.d) This kernal runs on a CPU platform.

(2.d) This runtime for this kernal is approximately 1.2 min (it took us 72.9 sec).

(3) Liverpool_Wavenet

(3.a) This last kernal outputs two csv files, the one named final_submission_wavenet.csv can be submitted to the competition to get a score close to our final private score.

(3.b) This kernal uses a Wavenet based model, which is trained on clean signal
and makes the final predictions.

(3.c) This kernal runs on a GPU platform.

(3.d) The runtime for this kernal is approximately 2.5 hrs (it took us 8908.9 sec)

Doing this iteratively, i.e generate better predictions and then generate cleaner 
signal and then use that cleaner signal to make even better prediction captures 
the basic strategy of our work.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train_data = pd.read_csv('/kaggle/input/data-without-drift/train_clean.csv')

In [ ]:
test_data = pd.read_csv('/kaggle/input/data-without-drift/test_clean.csv')

In [ ]:
test_data[700000:800000]['signal']=test_data[700000:800000]['signal'].values - train_data[4000000:4100000]['signal'].values

In [ ]:
train_oofs = pd.read_csv('/kaggle/input/liverpool-lgbm-oofs/oofs_train.csv')

In [ ]:
test_oofs = pd.read_csv('/kaggle/input/liverpool-lgbm-oofs/oofs_test.csv')

In [ ]:
test_data[200000:300000]['signal']=test_data[200000:300000]['signal'].values + 0.017

In [ ]:
test_data[500000:600000]['signal']=test_data[500000:600000]['signal'].values + 0.05

In [ ]:
train_data[2500000:3000000]['signal']=train_data[2500000:3000000]['signal'].values - 0.015

In [ ]:
train = pd.DataFrame()
train['signal'] = train_data['signal'].values
train['open_channels'] = train_oofs.oofs2.values
test = pd.DataFrame()
test['signal'] = test_data['signal'].values
test['open_channels'] = test_oofs.oofs2.values

In [ ]:
train['meanp']=train.groupby('open_channels').signal.transform('mean')
train['noisep']=-(train.meanp-train.signal)
test['meanp']=test.groupby('open_channels').signal.transform('mean')
test['noisep']=-(test.meanp-test.signal)

In [ ]:
train['phase']=train.index
train['phase']=train.phase%200
test['phase']=test.index
test['phase']=test.phase%200

In [ ]:
for i in range(0, 200):
    train.loc[train.phase==i,'ave_mod200']=train.loc[train.phase==i].noisep.rolling(window=10, min_periods=1, center=True).mean()
    test.loc[test.phase==i,'ave_mod200']=test.loc[test.phase==i].noisep.rolling(window=10, min_periods=1, center=True).mean()

In [ ]:
train['noise_intrabatch_with200ave']=train['ave_mod200'].rolling(window=3, min_periods=1, center=True).mean()
test['noise_intrabatch_with200ave']=test['ave_mod200'].rolling(window=3, min_periods=1, center=True).mean()

In [ ]:
train['clean_s']=train['signal'].values - train['noise_intrabatch_with200ave'].values
test['clean_s']=test['signal'].values - test['noise_intrabatch_with200ave'].values

In [ ]:
clean_train = pd.DataFrame()
clean_test = pd.DataFrame()
clean_train['signal']=train['clean_s'].values
clean_test['signal']=test['clean_s'].values
clean_train.to_csv('clean_train_signal.csv', index = False)
clean_test.to_csv('clean_test_signal.csv', index = False)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(test_data['time'], clean_test['signal'], linewidth=.1)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(test_data['time'], test_data['signal'], linewidth=.1)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(test_data['time'], test_data['signal'] - clean_test['signal'], linewidth=.1)

In [ ]:
plt.plot(train_data['time'], clean_train['signal'], linewidth=.1)

In [ ]:
plt.plot(train_data['time'], train_data['signal'], linewidth=.1)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_data['time'], train_data['signal'] - clean_train['signal'], linewidth=.1)